# Form Recognizer를 사용하여 영수증 분석

![영수증을 들고 있는 로봇](./images/receipt_analysis.jpg)

Computer Vision의 AI(인공 지능) 분야에서 OCR(광학 인식)은 인쇄된 문서나 필기 문서를 읽는 데 주로 사용됩니다. 종종 텍스트는 추가적인 처리 또는 분석에 사용할 수 있는 형식으로 문서에서 간단히 추출됩니다.

보다 진보된 OCR 시나리오는 양식의 필드가 나타내는 의미를 이해하면서 구매 주문서나 송장 같은 양식에서 정보를 추출하는 것입니다. **Form Recognizer** 서비스는 이러한 종류의 AI 문제를 위해 특별히 설계되었습니다.

## 영수증 보기

이 예에서는 Form Recognizer의 기본 제공 모델을 사용하여 영수증을 분석합니다.

아래의 **셀 실행**(&#9655;) 단추(셀 왼쪽에 있음)를 클릭하여 실행하고 Form Recognizer를 사용하여 분석할 영수증의 예를 확인해 보세요.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline


# Load and display a receipt image
fig = plt.figure(figsize=(6, 6))
image_path = os.path.join('data', 'form-receipt', 'receipt.jpg')
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

## Form Recognizer 리소스 만들기

먼저 Azure 구독에서 Form Recognizer 리소스를 만듭니다.

1. 다른 브라우저 탭에서 Azure Portal(https://portal.azure.com) 을 열고 Microsoft 계정으로 로그인합니다.
2. **+ 리소스 만들기**를 선택하고 *Form Recognizer*를 검색합니다.
3. 서비스 목록에서 **Form Recognizer**를 선택합니다.
4. **Form Recognizer** 블레이드에서 **만들기**를 선택합니다.
5. **만들기** 블레이드에서 다음 세부 정보를 입력하고 **만들기**를 선택합니다.
   - **이름**: 서비스의 고유한 이름
   - **구독**: 사용자의 Azure 구독
   - **지역**: 사용 가능한 영역
   - **가격 책정 계층**: F0
   - **리소스 그룹**: 이전에 사용한 기존 리소스 그룹
   - **아래 알림을 읽고 이해했음을 확인합니다**. 선택됨.
6. 서비스가 생성될 때까지 기다립니다.
7. Azure Portal에서 새로 생성된 Form Recognizer 서비스를 확인합니다. 그리고 **키 및 엔드포인트** 페이지에서 **Key1** 및 **엔드포인트** 값을 복사하고 아래 코드 셀에 붙여 넣어 **YOUR_FORM_KEY** 및 **YOUR_FORM_ENDPOINT**를 대체합니다.

In [ ]:
form_key = 'YOUR_FORM_KEY'
form_endpoint = 'YOUR_FORM_ENDPOINT'

print('Ready to use form recognizer at {} using key {}'.format(form_endpoint, form_key))

## 영수증 분석

이제 Form Recognizer를 사용하여 영수증을 분석할 준비가 되었습니다.

In [ ]:
import os
from azure.ai.formrecognizer import FormRecognizerClient
from azure.core.credentials import AzureKeyCredential

# Create a client for the form recognizer service
form_recognizer_client = FormRecognizerClient(endpoint=form_endpoint, credential=AzureKeyCredential(form_key))

try:
    print("Analyzing receipt...")
    # Get the receipt image file
    image_path = os.path.join('data', 'form-receipt', 'receipt.jpg')

    # Submit the file data to form recognizer
    with open(image_path, "rb") as f:
        analyze_receipt = form_recognizer_client.begin_recognize_receipts(receipt=f)
    
    # Get the results
    receipt_data = analyze_receipt.result()

    # Print the extracted data for the first (and only) receipt
    receipt = receipt_data[0]
    receipt_type = receipt.fields.get("ReceiptType")
    if receipt_type:
        print("Receipt Type: {}".format(receipt_type.value))
    merchant_address = receipt.fields.get("MerchantAddress")
    if merchant_address:
        print("Merchant Address: {}".format(merchant_address.value))
    merchant_phone = receipt.fields.get("MerchantPhoneNumber")
    if merchant_phone:
        print("Merchant Phone: {}".format(merchant_phone.value))
    transaction_date = receipt.fields.get("TransactionDate")
    if transaction_date:
        print("Transaction Date: {}".format(transaction_date.value))
    print("Receipt items:")
    items = receipt.fields.get("Items")
    if items:
        for idx, item in enumerate(receipt.fields.get("Items").value):
            print("\tItem #{}".format(idx+1))
            item_name = item.value.get("Name")
            if item_name:
                print("\t - Name: {}".format(item_name.value))
            item_total_price = item.value.get("TotalPrice")
            if item_total_price:
                print("\t - Price: {}".format(item_total_price.value))
    subtotal = receipt.fields.get("Subtotal")
    if subtotal:
        print("Subtotal: {} ".format(subtotal.value))
    tax = receipt.fields.get("Tax")
    if tax:
        print("Tax: {}".format(tax.value))
    total = receipt.fields.get("Total")
    if total:
        print("Total: {}".format(total.value))

except Exception as ex:
    print('Error:', ex)


Form Recognizer는 양식에 있는 데이터를 해석하여 가맹점 주소 및 전화번호, 거래 날짜 및 시간, 품목명, 소계, 세금, 총액을 올바르게 식별할 수 있습니다.

## 추가 정보

Form Recognizer 서비스에 대한 자세한 내용은 [Form Recognizer 설명서](https://docs.microsoft.com/ko-kr/azure/cognitive-services/form-recognizer/index)를 참조하세요.